In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle

# for one hot encoding with feature-engine
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.impute import SimpleImputer

#for training and imputing missing values
from sklearn.ensemble import RandomForestClassifier
import joblib
from MultiColumnLabelEncoder import MultiColumnLabelEncoder
%matplotlib inline

In [2]:
df=pd.read_csv('incident_event_log.csv')

In [3]:
df.head(5)

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   caller_id                141712 non-null  object
 8   opened_by                141712 non-null  object
 9   opened_at                141712 non-null  object
 10  sys_created_by           141712 non-null  object
 11  sys_created_at           141712 non-null  object
 12  sys_updated_by           141712 non-null  object
 13  sys_updated_at           141712 non-null  object
 14  contact_type        

<h3>Findings from EDA</h3>
As we found in the EDA steps that below features have most of the values as ?, hence we will not be using them for building the ML model. 
<ol>
    <li>cmdb_ci</li>
    <li>problem_id</li>
    <li>rfc</li>
    <li>vendor</li>
    <li>caused_by</li>
    <li>resolved_at</li>
    <li>sys_created_at</li>
</ol>

In [5]:
df = df.drop(['cmdb_ci','problem_id','rfc','vendor','caused_by', 'resolved_at', 'sys_created_at'], axis=1)    

<h3>Since the data various types of features - Number, Categorical, Temporal, Mixed. <br/>We need to process each feature differently.</h3>

In [6]:
categorical_non_ordinal_features = ['incident_state', 'active', 'made_sla','contact_type', 'knowledge','u_priority_confirmation'
                                    , 'notify']
categorical_ordinal_features = [ 'impact', 'urgency', 'priority']
categorical_features = categorical_non_ordinal_features+categorical_ordinal_features
numerical_features = ['reassignment_count', 'reopen_count', 'sys_mod_count']
temporal_features = ['opened_at','sys_updated_at', 'closed_at']
mixed_features = ['number','caller_id', 'sys_created_by','opened_by','sys_updated_by', 'location',
                  'category', 'subcategory', 'u_symptom', 'assignment_group', 'assigned_to', 'closed_code', 'resolved_by']

In [7]:
# https://stackoverflow.com/a/49406417/1225413
df=df.replace({'?': None})

In [8]:
# https://stackoverflow.com/a/36849011/1225413
# https://stackoverflow.com/a/2073189/1225413
date_format='%d/%m/%Y %H:%M'
df['opened_at'] = pd.to_datetime(df.opened_at, format=date_format)
df['sys_updated_at'] = pd.to_datetime(df.sys_updated_at, format=date_format)
df['closed_at'] = pd.to_datetime(df.closed_at, format=date_format)

In [9]:
df['closed_at'] 

0        2016-03-05 12:00:00
1        2016-03-05 12:00:00
2        2016-03-05 12:00:00
3        2016-03-05 12:00:00
4        2016-03-06 10:00:00
                 ...        
141707   2017-02-16 09:53:00
141708   2017-02-16 16:38:00
141709   2017-02-16 16:38:00
141710   2017-02-16 16:38:00
141711   2017-02-16 16:38:00
Name: closed_at, Length: 141712, dtype: datetime64[ns]

In [10]:
df['opened_at']

0        2016-02-29 01:16:00
1        2016-02-29 01:16:00
2        2016-02-29 01:16:00
3        2016-02-29 01:16:00
4        2016-02-29 04:40:00
                 ...        
141707   2017-02-16 09:09:00
141708   2017-02-16 14:17:00
141709   2017-02-16 14:17:00
141710   2017-02-16 14:17:00
141711   2017-02-16 14:17:00
Name: opened_at, Length: 141712, dtype: datetime64[ns]

In [11]:
df['time_taken_to_complete'] = (df['closed_at'] - df['opened_at'])/ pd.Timedelta(days=1)
df[['time_taken_to_complete']]

,time_taken_to_complete
0,5.447222
1,5.447222
2,5.447222
3,5.447222
4,6.222222
...,...
141707,0.030556
141708,0.097917
141709,0.097917
141710,0.097917


<h1>Imputing Categorical Features</h1>

In [12]:
for col in categorical_features:
    print("Feature -\t",col)
    print("-"*20)
    print(df[col].value_counts())
    print("-"*20)

Feature -	 incident_state
--------------------
Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
-100                      5
Name: incident_state, dtype: int64
--------------------
Feature -	 active
--------------------
True     116726
False     24986
Name: active, dtype: int64
--------------------
Feature -	 made_sla
--------------------
True     132497
False      9215
Name: made_sla, dtype: int64
--------------------
Feature -	 contact_type
--------------------
Phone             140462
Self service         995
Email                220
IVR                   18
Direct opening        17
Name: contact_type, dtype: int64
--------------------
Feature -	 knowledge
--------------------
False    116349
True      25363
Name: knowledge, dtype: int64
--------------------
Feature -	 u_priority_confirmation
-------------------

<h3> Observation:</h3>
Only the feature incident_state have -100 value, which we need to impute. 

In [13]:
mask = df.incident_state=="-100"
column_name = 'incident_state'
df.loc[mask, column_name] = np.NAN

In [14]:
df[df["incident_state"]=="-100"]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,closed_at,time_taken_to_complete


In [15]:
df[df["incident_state"]==-100]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,closed_at,time_taken_to_complete


In [16]:
df[df["incident_state"].isna()]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,closed_at,time_taken_to_complete
115641,INC0028089,NaN,True,0,0,1,True,Caller 290,Opened by 305,2016-05-10 22:09:00,...,3 - Moderate,Group 64,Resolver 15,False,False,Do Not Notify,code 7,Resolved by 175,2016-06-21 13:07:00,41.623611
115642,INC0028089,NaN,True,0,0,2,True,Caller 290,Opened by 305,2016-05-10 22:09:00,...,3 - Moderate,Group 64,Resolver 15,False,False,Do Not Notify,code 7,Resolved by 175,2016-06-21 13:07:00,41.623611
124507,INC0030413,NaN,True,1,0,14,True,Caller 2950,Opened by 17,2016-05-17 14:42:00,...,3 - Moderate,Group 17,Resolver 77,False,False,Do Not Notify,code 7,Resolved by 70,2016-06-26 18:00:00,40.137500
124508,INC0030413,NaN,True,1,0,15,True,Caller 2950,Opened by 17,2016-05-17 14:42:00,...,3 - Moderate,Group 17,Resolver 77,False,False,Do Not Notify,code 7,Resolved by 70,2016-06-26 18:00:00,40.137500
124509,INC0030413,NaN,True,1,0,20,True,Caller 2950,Opened by 17,2016-05-17 14:42:00,...,3 - Moderate,Group 17,Resolver 77,False,False,Do Not Notify,code 7,Resolved by 70,2016-06-26 18:00:00,40.137500


In [17]:
# https://stackoverflow.com/a/49806113/1225413
'''Since we have -100 in incident state, we are imputing with most frequent value'''
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.NAN, strategy="most_frequent")
df["incident_state"] = imp.fit_transform(df[["incident_state"]]).ravel()

In [18]:
df[df["incident_state"].isna()]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,closed_at,time_taken_to_complete


In [19]:
filename = "./git_repo/saved_models/incident_state_simple_imputer"+'.pkl'
# joblib.dump(imp, filename, compress=3)
pickle.dump(imp, open(filename, 'wb'))

In [20]:
df[df["incident_state"]=="-100"]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,closed_at,time_taken_to_complete


In [21]:
list(df.columns)

['number',
 'incident_state',
 'active',
 'reassignment_count',
 'reopen_count',
 'sys_mod_count',
 'made_sla',
 'caller_id',
 'opened_by',
 'opened_at',
 'sys_created_by',
 'sys_updated_by',
 'sys_updated_at',
 'contact_type',
 'location',
 'category',
 'subcategory',
 'u_symptom',
 'impact',
 'urgency',
 'priority',
 'assignment_group',
 'assigned_to',
 'knowledge',
 'u_priority_confirmation',
 'notify',
 'closed_code',
 'resolved_by',
 'closed_at',
 'time_taken_to_complete']

In [22]:
df.iloc[:,[1]]

,incident_state
0,New
1,Resolved
2,Resolved
3,Closed
4,New
...,...
141707,Closed
141708,Active
141709,Active
141710,Resolved


In [23]:
df["reassignment_count"].dtype

dtype('int64')

<h1>Cleaning mixed features(features which have both alphabets and numbers)</h1>

In [24]:
for col in mixed_features:
    totalNull = df[col].isnull().sum()
    if totalNull>0 :        
        print(col," has ",totalNull," empty values")       

caller_id  has  29  empty values
sys_created_by  has  53076  empty values
opened_by  has  4835  empty values
location  has  76  empty values
category  has  78  empty values
subcategory  has  111  empty values
u_symptom  has  32964  empty values
assignment_group  has  14213  empty values
assigned_to  has  27496  empty values
closed_code  has  714  empty values
resolved_by  has  226  empty values


'''Capturing numerical part for number feature'''
# https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
# https://stackoverflow.com/a/38706893/1225413
df["incident_number"] = df["number"].str.replace("INC", "")

# https://www.geeksforgeeks.org/python-pandas-dataframe-astype/
df['incident_number']=df['incident_number'].astype("int64")

In [25]:
'''Capturing numerical part for number feature'''
# https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
# https://stackoverflow.com/a/38706893/1225413
df["incident_number"] = df["number"].str.replace("INC", "")

In [26]:
'''Capturing numerical part for caller_id feature'''
df["caller_id_num"] = df["caller_id"].str.replace("Caller", "")

df[["caller_id_num"]] = df[["caller_id_num"]].fillna(0)

In [27]:
'''Capturing numerical part for sys_created_by feature'''
df["sys_created_by_num"] = df["sys_created_by"].str.replace("Created by", "")

df[["sys_created_by_num"]] = df[["sys_created_by_num"]].fillna(0)

In [28]:
'''Capturing numerical part for sys_updated_by feature'''
df["sys_updated_by_num"] = df["sys_updated_by"].str.replace("Updated by", "")

df[["sys_updated_by_num"]] = df[["sys_updated_by_num"]].fillna(0)

In [29]:
'''Capturing numerical part for opened_by feature'''
df["opened_by_num"] = df["opened_by"].str.replace("Opened by", "")

df[["opened_by_num"]] = df[["opened_by_num"]].fillna(0)

In [30]:
'''Capturing numerical part for location feature'''
df["location_num"] = df["location"].str.replace("Location", "")

df[["location_num"]] = df[["location_num"]].fillna(0)

In [31]:
'''Capturing numerical part for category feature'''
df["category_num"] = df["category"].str.replace("Category", "")

df[["category_num"]] = df[["category_num"]].fillna(0)

In [32]:
'''Capturing numerical part for subcategory feature'''
df["subcategory_num"] = df["subcategory"].str.replace("Subcategory", "")

df[["subcategory_num"]] = df[["subcategory_num"]].fillna(0)

In [33]:
'''Capturing numerical part for u_symptom feature'''
df["u_symptom_num"] = df["u_symptom"].str.replace("Symptom", "")

df[["u_symptom_num"]] = df[["u_symptom_num"]].fillna(0)

In [34]:
'''Capturing numerical part for assignment_group feature'''
df["assignment_group_num"] = df["assignment_group"].str.replace("Group", "")

df[["assignment_group_num"]] = df[["assignment_group_num"]].fillna(0)

In [35]:
'''Capturing numerical part for assigned_to feature'''
df["assigned_to_num"] = df["assigned_to"].str.replace("Resolver", "")

df[["assigned_to_num"]] = df[["assigned_to_num"]].fillna(0)

In [36]:
'''Capturing numerical part for closed_code feature'''
df["closed_code_num"] = df["closed_code"].str.replace("code", "")

df[["closed_code_num"]] = df[["closed_code_num"]].fillna(0)

In [37]:
'''Capturing numerical part for resolved_by feature'''
df["resolved_by_num"] = df["resolved_by"].str.replace("Resolved by", "")

df[["resolved_by_num"]] = df[["resolved_by_num"]].fillna(0)

In [38]:
df.head(5)

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,sys_updated_by_num,opened_by_num,location_num,category_num,subcategory_num,u_symptom_num,assignment_group_num,assigned_to_num,closed_code_num,resolved_by_num
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,21,8,143,55,170,72,56,0,5,149
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,642,8,143,55,170,72,56,0,5,149
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,804,8,143,55,170,72,56,0,5,149
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,908,8,143,55,170,72,56,0,5,149
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,...,746,397,165,40,215,471,70,89,5,81


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 43 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   number                   141712 non-null  object        
 1   incident_state           141712 non-null  object        
 2   active                   141712 non-null  bool          
 3   reassignment_count       141712 non-null  int64         
 4   reopen_count             141712 non-null  int64         
 5   sys_mod_count            141712 non-null  int64         
 6   made_sla                 141712 non-null  bool          
 7   caller_id                141683 non-null  object        
 8   opened_by                136877 non-null  object        
 9   opened_at                141712 non-null  datetime64[ns]
 10  sys_created_by           88636 non-null   object        
 11  sys_updated_by           141712 non-null  object        
 12  sys_updated_at  

<h1>Feature Engineering</h1>

In [40]:
df.to_csv(path_or_buf="incident_log_before_split.csv")

In [41]:
incident_log_before_split_filehandler = open("incident_log_before_split.pkl","wb")
pickle.dump(df,incident_log_before_split_filehandler)
incident_log_before_split_filehandler.close()

In [42]:
incident_log_before_split_filehandler = open("incident_log_before_split.pkl","rb")
df=pickle.load(incident_log_before_split_filehandler)
incident_log_before_split_filehandler.close()

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['time_taken_to_complete'], axis=1),  # predictors
    df['time_taken_to_complete'],  # target
    test_size=0.3, 
    random_state=0)  

X_train.shape, X_test.shape

((99198, 42), (42514, 42))

In [44]:
df

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,sys_updated_by_num,opened_by_num,location_num,category_num,subcategory_num,u_symptom_num,assignment_group_num,assigned_to_num,closed_code_num,resolved_by_num
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,21,8,143,55,170,72,56,0,5,149
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,642,8,143,55,170,72,56,0,5,149
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,804,8,143,55,170,72,56,0,5,149
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,908,8,143,55,170,72,56,0,5,149
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,2016-02-29 04:40:00,...,746,397,165,40,215,471,70,89,5,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141707,INC0120835,Closed,False,1,0,4,True,Caller 116,Opened by 12,2017-02-16 09:09:00,...,27,12,204,42,223,494,31,10,9,9
141708,INC0121064,Active,True,0,0,0,True,Caller 116,Opened by 12,2017-02-16 14:17:00,...,908,12,204,42,223,494,70,10,6,9
141709,INC0121064,Active,True,1,0,1,True,Caller 116,Opened by 12,2017-02-16 14:17:00,...,60,12,204,42,223,494,31,0,6,9
141710,INC0121064,Resolved,True,1,0,2,True,Caller 116,Opened by 12,2017-02-16 14:17:00,...,27,12,204,42,223,494,31,10,6,9


<h1>Engineer Categorical Features</h1>

In [45]:
df.loc[df["caller_id_num"]==0]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,sys_updated_by_num,opened_by_num,location_num,category_num,subcategory_num,u_symptom_num,assignment_group_num,assigned_to_num,closed_code_num,resolved_by_num
25376,INC0005690,New,True,0,0,0,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,494,36,161,8,145
25377,INC0005690,New,True,0,0,1,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,491,36,161,8,145
25378,INC0005690,New,True,1,0,2,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,0,70,161,8,145
25379,INC0005690,Active,True,2,0,3,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,491,36,161,8,145
25380,INC0005690,Active,True,2,0,4,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,491,36,161,8,145
25381,INC0005690,Resolved,True,2,0,5,True,None,Opened by 337,2016-03-11 03:36:00,...,703,337,71,40,174,491,36,161,8,145
25382,INC0005690,Closed,False,2,0,6,True,None,Opened by 337,2016-03-11 03:36:00,...,823,337,71,40,174,491,36,161,8,145
54161,INC0012463,New,True,0,0,0,True,None,None,2016-03-29 11:12:00,...,370,0,188,46,223,592,55,194,6,177
54162,INC0012463,New,True,0,0,1,True,None,None,2016-03-29 11:12:00,...,277,0,188,46,223,592,55,194,6,177
54163,INC0012463,New,True,1,0,2,True,None,None,2016-03-29 11:12:00,...,277,0,188,46,223,592,70,194,6,177


In [46]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
# https://stackoverflow.com/a/56739889/1225413
non_ordinal_ohe = OneHotEncoder(drop='first', dtype='int',sparse=False)
non_ordinal_ohe.fit(X_train[categorical_non_ordinal_features])

OneHotEncoder(categories='auto', drop='first', dtype='int',
              handle_unknown='error', sparse=False)

In [47]:
non_ordinal_ohe_filehandler = open("./git_repo/saved_models/non_ordinal_ohe.pkl","wb")
pickle.dump(non_ordinal_ohe,non_ordinal_ohe_filehandler)
non_ordinal_ohe_filehandler.close()

In [48]:
# https://stackoverflow.com/a/63480537/1225413
# https://stackoverflow.com/a/56739889/1225413
temp=non_ordinal_ohe.transform(X_train[categorical_non_ordinal_features])
one_hot_encoded_column_names = non_ordinal_ohe.get_feature_names(categorical_non_ordinal_features)
one_hot_encoded_df =  pd.DataFrame(temp, columns= one_hot_encoded_column_names, index=X_train.index)

In [49]:
X_train = pd.concat([X_train, one_hot_encoded_df], axis='columns')
X_train = X_train.drop(columns=categorical_non_ordinal_features)

In [50]:
temp=non_ordinal_ohe.transform(X_test[categorical_non_ordinal_features])
one_hot_encoded_column_names = non_ordinal_ohe.get_feature_names(categorical_non_ordinal_features)
one_hot_encoded_df =  pd.DataFrame(temp, columns= one_hot_encoded_column_names, index=X_test.index)
X_test = pd.concat([X_test, one_hot_encoded_df], axis='columns')
X_test = X_test.drop(columns=categorical_non_ordinal_features)

In [51]:
X_train.head(5)

,number,reassignment_count,reopen_count,sys_mod_count,caller_id,opened_by,opened_at,sys_created_by,sys_updated_by,sys_updated_at,...,incident_state_Resolved,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email
12846,INC0002983,0,0,0,Caller 2001,Opened by 108,2016-03-06 17:40:00,Created by 52,Updated by 214,2016-03-06 17:45:00,...,0,1,1,0,0,1,0,1,0,0
16096,INC0003575,0,0,0,Caller 2895,Opened by 59,2016-03-07 13:56:00,None,Updated by 908,2016-03-07 13:56:00,...,0,1,1,0,0,1,0,1,0,0
52740,INC0012122,1,0,7,Caller 1727,Opened by 24,2016-03-28 15:40:00,None,Updated by 908,2016-04-13 15:59:00,...,0,0,0,0,0,1,0,0,1,0
135624,INC0033264,1,1,8,Caller 4131,Opened by 305,2016-05-26 21:12:00,None,Updated by 745,2016-05-27 18:21:00,...,0,1,1,0,0,1,0,0,1,0
55379,INC0012794,2,0,7,Caller 2691,Opened by 40,2016-03-30 07:49:00,Created by 20,Updated by 777,2016-04-12 16:45:00,...,0,1,1,0,0,1,0,0,0,0


In [52]:
X_train["impact"]

12846     2 - Medium
16096     2 - Medium
52740     2 - Medium
135624    2 - Medium
55379     2 - Medium
             ...    
41993     2 - Medium
97639     2 - Medium
95939     2 - Medium
117952    2 - Medium
43567     2 - Medium
Name: impact, Length: 99198, dtype: object

In [53]:
label_encoder_impact = LabelEncoder()
label_encoder_urgency = LabelEncoder()
label_encoder_priority = LabelEncoder()

label_encoder_impact.fit(X_train['impact'])
label_encoder_urgency.fit(X_train['urgency'])
label_encoder_priority.fit(X_train['priority'])

X_train['impact'] = label_encoder_impact.transform(X_train['impact'])
X_train['urgency'] = label_encoder_urgency.transform(X_train['urgency'])
X_train['priority'] = label_encoder_priority.transform(X_train['priority'])

X_test['impact'] = label_encoder_impact.transform(X_test['impact'])
X_test['urgency'] = label_encoder_urgency.transform(X_test['urgency'])
X_test['priority'] = label_encoder_priority.transform(X_test['priority'])

In [54]:
X_test['impact'].value_counts()

1    40207
2     1194
0     1113
Name: impact, dtype: int64

In [55]:
label_encoder_impact_filehandler = open("./git_repo/saved_models/label_encoder_impact.pkl","wb")
pickle.dump(label_encoder_impact,label_encoder_impact_filehandler)
label_encoder_impact_filehandler.close()

label_encoder_urgency_filehandler = open("./git_repo/saved_models/label_encoder_urgency.pkl","wb")
pickle.dump(label_encoder_urgency,label_encoder_urgency_filehandler)
label_encoder_urgency_filehandler.close()

label_encoder_priority_filehandler = open("./git_repo/saved_models/label_encoder_priority.pkl","wb")
pickle.dump(label_encoder_priority,label_encoder_priority_filehandler)
label_encoder_priority_filehandler.close()

<h1>Engineer Temporal Featues</h1>

In [56]:
'''Not doing opened_at and closed_at features as they will removed as it is'''
# https://stackoverflow.com/a/34883876/1225413
X_train['sys_updated_at_ms'] = pd.to_datetime(X_train['sys_updated_at'], unit='ms').astype(np.int64)

X_test['sys_updated_at_ms'] = pd.to_datetime(X_test['sys_updated_at'], unit='ms').astype(np.int64)

C:\Anaconda3\envs\tfg2.4\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\envs\tfg2.4\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """


In [57]:
X_test

,number,reassignment_count,reopen_count,sys_mod_count,caller_id,opened_by,opened_at,sys_created_by,sys_updated_by,sys_updated_at,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
83151,INC0019911,1,0,4,Caller 1932,Opened by 386,2016-04-15 15:45:00,Created by 167,Updated by 241,2016-04-18 09:26:00,...,1,1,0,0,1,0,0,0,0,1460971560000000000
101627,INC0024642,1,0,1,Caller 106,Opened by 501,2016-05-02 08:24:00,Created by 216,Updated by 915,2016-05-02 08:28:00,...,1,1,0,0,1,0,0,0,0,1462177680000000000
64377,INC0015007,0,0,9,Caller 2632,Opened by 17,2016-04-04 15:27:00,Created by 10,Updated by 109,2016-04-08 09:11:00,...,1,1,0,0,1,0,0,0,0,1460106660000000000
87520,INC0020962,1,0,1,Caller 4120,Opened by 239,2016-04-19 14:20:00,Created by 107,Updated by 467,2016-04-19 14:21:00,...,1,1,0,0,1,0,0,0,0,1461075660000000000
128434,INC0031416,0,0,0,Caller 2651,Opened by 94,2016-05-20 00:48:00,None,Updated by 908,2016-05-20 00:48:00,...,1,1,0,0,1,0,0,0,0,1463705280000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28230,INC0006392,2,0,6,Caller 4767,Opened by 397,2016-03-14 08:12:00,Created by 171,Updated by 186,2016-03-15 12:04:00,...,1,1,0,0,1,0,0,0,0,1458043440000000000
103960,INC0025199,0,0,0,Caller 2672,Opened by 17,2016-05-03 08:11:00,Created by 10,Updated by 44,2016-05-03 08:19:00,...,1,1,0,0,1,0,0,0,0,1462263540000000000
23949,INC0005312,1,0,12,Caller 3728,Opened by 17,2016-03-10 10:44:00,Created by 10,Updated by 695,2016-04-06 19:07:00,...,1,1,0,0,1,0,1,0,0,1459969620000000000
63917,INC0014899,0,0,1,Caller 2447,Opened by 17,2016-04-04 13:18:00,None,Updated by 44,2016-04-04 15:09:00,...,1,1,0,0,1,0,0,1,0,1459782540000000000


<h1>Engineer Numerical Features</h1>

In [58]:
X_train[numerical_features]

,reassignment_count,reopen_count,sys_mod_count
12846,0,0,0
16096,0,0,0
52740,1,0,7
135624,1,1,8
55379,2,0,7
...,...,...,...
41993,1,0,6
97639,1,0,4
95939,0,0,0
117952,0,0,2


In [59]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train[numerical_features])
X_train[numerical_features]=scaler.transform(X_train[numerical_features])

In [60]:
X_train[numerical_features]

,reassignment_count,reopen_count,sys_mod_count
12846,-0.636268,-0.105307,-0.658175
16096,-0.636268,-0.105307,-0.658175
52740,-0.059340,-0.105307,0.247074
135624,-0.059340,4.695363,0.376395
55379,0.517588,-0.105307,0.247074
...,...,...,...
41993,-0.059340,-0.105307,0.117753
97639,-0.059340,-0.105307,-0.140890
95939,-0.636268,-0.105307,-0.658175
117952,-0.636268,-0.105307,-0.399533


In [61]:
numerical_standard_scaler_filehandler = open("./git_repo/saved_models/numerical_standard_scaler.pkl","wb")
pickle.dump(scaler,numerical_standard_scaler_filehandler)
numerical_standard_scaler_filehandler.close()

In [62]:
X_test[numerical_features]=scaler.transform(X_test[numerical_features])

In [63]:
X_test

,number,reassignment_count,reopen_count,sys_mod_count,caller_id,opened_by,opened_at,sys_created_by,sys_updated_by,sys_updated_at,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
83151,INC0019911,-0.059340,-0.105307,-0.140890,Caller 1932,Opened by 386,2016-04-15 15:45:00,Created by 167,Updated by 241,2016-04-18 09:26:00,...,1,1,0,0,1,0,0,0,0,1460971560000000000
101627,INC0024642,-0.059340,-0.105307,-0.528854,Caller 106,Opened by 501,2016-05-02 08:24:00,Created by 216,Updated by 915,2016-05-02 08:28:00,...,1,1,0,0,1,0,0,0,0,1462177680000000000
64377,INC0015007,-0.636268,-0.105307,0.505717,Caller 2632,Opened by 17,2016-04-04 15:27:00,Created by 10,Updated by 109,2016-04-08 09:11:00,...,1,1,0,0,1,0,0,0,0,1460106660000000000
87520,INC0020962,-0.059340,-0.105307,-0.528854,Caller 4120,Opened by 239,2016-04-19 14:20:00,Created by 107,Updated by 467,2016-04-19 14:21:00,...,1,1,0,0,1,0,0,0,0,1461075660000000000
128434,INC0031416,-0.636268,-0.105307,-0.658175,Caller 2651,Opened by 94,2016-05-20 00:48:00,None,Updated by 908,2016-05-20 00:48:00,...,1,1,0,0,1,0,0,0,0,1463705280000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28230,INC0006392,0.517588,-0.105307,0.117753,Caller 4767,Opened by 397,2016-03-14 08:12:00,Created by 171,Updated by 186,2016-03-15 12:04:00,...,1,1,0,0,1,0,0,0,0,1458043440000000000
103960,INC0025199,-0.636268,-0.105307,-0.658175,Caller 2672,Opened by 17,2016-05-03 08:11:00,Created by 10,Updated by 44,2016-05-03 08:19:00,...,1,1,0,0,1,0,0,0,0,1462263540000000000
23949,INC0005312,-0.059340,-0.105307,0.893681,Caller 3728,Opened by 17,2016-03-10 10:44:00,Created by 10,Updated by 695,2016-04-06 19:07:00,...,1,1,0,0,1,0,1,0,0,1459969620000000000
63917,INC0014899,-0.636268,-0.105307,-0.528854,Caller 2447,Opened by 17,2016-04-04 13:18:00,None,Updated by 44,2016-04-04 15:09:00,...,1,1,0,0,1,0,0,1,0,1459782540000000000


In [64]:
'''Generic method to save object in a file using pickle.'''
def dump_object_to_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj,f)

<h1>Engineer Numericalized Temporal Features</h1>

In [65]:
df.loc[:0]

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,sys_updated_by_num,opened_by_num,location_num,category_num,subcategory_num,u_symptom_num,assignment_group_num,assigned_to_num,closed_code_num,resolved_by_num
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,21,8,143,55,170,72,56,0,5,149


In [66]:
temporal_number_features=["sys_updated_at_ms"]

In [67]:
temporal_number_scaler = StandardScaler()
temporal_number_scaler.fit(X_train[temporal_number_features])
X_train[temporal_number_features]=temporal_number_scaler.transform(X_train[temporal_number_features])
X_test[temporal_number_features]=temporal_number_scaler.transform(X_test[temporal_number_features])

In [68]:
dump_object_to_pickle(temporal_number_scaler, "./git_repo/saved_models/temporal_number_scaler.pkl")

In [69]:
X_train.index

Int64Index([ 12846,  16096,  52740, 135624,  55379,  47879,  71470,  96284,
             26889,  79534,
            ...
             52620,  48600, 112420,  86293, 122579,  41993,  97639,  95939,
            117952,  43567],
           dtype='int64', length=99198)

In [70]:
X_train

,number,reassignment_count,reopen_count,sys_mod_count,caller_id,opened_by,opened_at,sys_created_by,sys_updated_by,sys_updated_at,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
12846,INC0002983,-0.636268,-0.105307,-0.658175,Caller 2001,Opened by 108,2016-03-06 17:40:00,Created by 52,Updated by 214,2016-03-06 17:45:00,...,1,1,0,0,1,0,1,0,0,-1.076930
16096,INC0003575,-0.636268,-0.105307,-0.658175,Caller 2895,Opened by 59,2016-03-07 13:56:00,None,Updated by 908,2016-03-07 13:56:00,...,1,1,0,0,1,0,1,0,0,-1.056181
52740,INC0012122,-0.059340,-0.105307,0.247074,Caller 1727,Opened by 24,2016-03-28 15:40:00,None,Updated by 908,2016-04-13 15:59:00,...,0,0,0,0,1,0,0,1,0,-0.141211
135624,INC0033264,-0.059340,4.695363,0.376395,Caller 4131,Opened by 305,2016-05-26 21:12:00,None,Updated by 745,2016-05-27 18:21:00,...,1,1,0,0,1,0,0,1,0,0.946789
55379,INC0012794,0.517588,-0.105307,0.247074,Caller 2691,Opened by 40,2016-03-30 07:49:00,Created by 20,Updated by 777,2016-04-12 16:45:00,...,1,1,0,0,1,0,0,0,0,-0.165095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,INC0009436,-0.059340,-0.105307,0.117753,Caller 3149,Opened by 17,2016-03-21 12:00:00,None,Updated by 16,2016-03-22 17:44:00,...,1,1,0,0,1,0,0,0,0,-0.682195
97639,INC0023611,-0.059340,-0.105307,-0.140890,Caller 4416,Opened by 17,2016-04-28 10:37:00,None,Updated by 915,2016-04-28 12:58:00,...,1,1,0,0,1,0,0,0,0,0.225768
95939,INC0023171,-0.636268,-0.105307,-0.658175,Caller 4713,Opened by 239,2016-04-27 11:52:00,None,Updated by 908,2016-04-27 11:52:00,...,1,1,0,0,1,0,0,0,0,0.199965
117952,INC0028681,-0.636268,-0.105307,-0.399533,Caller 4512,Opened by 460,2016-05-12 09:14:00,Created by 199,Updated by 908,2016-05-17 10:07:00,...,0,1,0,0,1,0,0,1,0,0.691605


<hr/>

<h1>Imputing Features by Training on Model</h1>

In [71]:
X_train = X_train.drop(columns=["caller_id", "number", "opened_by", "sys_created_by",
              "sys_updated_by","location","category","subcategory", "u_symptom","assignment_group",
              "assigned_to","closed_code","resolved_by", "incident_number",'opened_at','sys_updated_at', 'closed_at'])

In [72]:
X_train.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_created_by_num,sys_updated_by_num,opened_by_num,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
12846,-0.636268,-0.105307,-0.658175,1,1,2,2001,52,214,108,...,1,1,0,0,1,0,1,0,0,-1.076930
16096,-0.636268,-0.105307,-0.658175,1,1,2,2895,0,908,59,...,1,1,0,0,1,0,1,0,0,-1.056181
52740,-0.059340,-0.105307,0.247074,1,1,2,1727,0,908,24,...,0,0,0,0,1,0,0,1,0,-0.141211
135624,-0.059340,4.695363,0.376395,1,1,2,4131,0,745,305,...,1,1,0,0,1,0,0,1,0,0.946789
55379,0.517588,-0.105307,0.247074,1,1,2,2691,20,777,40,...,1,1,0,0,1,0,0,0,0,-0.165095


In [73]:
X_test = X_test.drop(columns=["caller_id", "number", "opened_by", "sys_created_by",
              "sys_updated_by","location","category","subcategory", "u_symptom","assignment_group",
              "assigned_to","closed_code","resolved_by", "incident_number",'opened_at','sys_updated_at', 'closed_at'])

In [74]:
mixed_features_engineered_columns= ["caller_id_num", "sys_created_by_num","sys_updated_by_num",
                                    "opened_by_num", "location_num","category_num","subcategory_num",
                                    "u_symptom_num","assignment_group_num","assigned_to_num",
                                    "closed_code_num","resolved_by_num"]

In [75]:
def print_columns_with_zero_values(df, column_list):
    for col in column_list:
        zero_counts=(df.loc[df[col]==0].shape[0])
        if zero_counts>0 :        
            print(col," has ",zero_counts," zero values")  

In [76]:
missing_data_df=[X_train]

In [77]:
missing_data_df = pd.concat(missing_data_df)

In [78]:
print("Missing dataframe features with count of zero values")
print("-"*50)
print_columns_with_zero_values(missing_data_df, mixed_features_engineered_columns)

Missing dataframe features with count of zero values
--------------------------------------------------
caller_id_num  has  19  zero values
sys_created_by_num  has  37103  zero values
opened_by_num  has  3427  zero values
location_num  has  50  zero values
category_num  has  50  zero values
subcategory_num  has  73  zero values
u_symptom_num  has  23139  zero values
assignment_group_num  has  10027  zero values
assigned_to_num  has  19139  zero values
closed_code_num  has  492  zero values
resolved_by_num  has  153  zero values


<h3>Observation:</h3>

The columns which have greater than 3% of values as 0 we will impute those columns by training. These are set of features we will be importing by training - sys_created_by_num, u_symptom_num, opened_by_num, assignment_group_num, assigned_to_num.

In [79]:
cols_to_be_imputed_using_training=["sys_created_by_num", "u_symptom_num", "opened_by_num", 
                                   "assignment_group_num", "assigned_to_num"]

In [80]:
def impute_feature_by_training(dataframe, column_to_be_imputed):
    
    #converting to int type from object
    dataframe[column_to_be_imputed] = dataframe[column_to_be_imputed].astype('int')
    
    #splitting dataset into train and test based on zero value
    reduced_train_df=dataframe.loc[missing_data_df[column_to_be_imputed]!=0]
    reduced_test_df=dataframe.loc[missing_data_df[column_to_be_imputed]==0]
    
    reduced_y_train=reduced_train_df[column_to_be_imputed]
    reduced_y_test=reduced_test_df[column_to_be_imputed]

    reduced_train_df=reduced_train_df.drop(columns=[column_to_be_imputed])
    reduced_test_df=reduced_test_df.drop(columns=[column_to_be_imputed])
    
    #training random forest classifer 
    rfc = RandomForestClassifier(n_jobs=1,max_depth=16)
    rfc.fit(reduced_train_df, reduced_y_train)
    
    y_pred=rfc.predict(reduced_test_df)
    
    # https://joblib.readthedocs.io/en/latest/generated/joblib.dump.html
    filename = column_to_be_imputed+"_rfc"+'.sav'
    joblib.dump(rfc, filename, compress=3)
    
    #merging back predicted values into original data frame
    reduced_test_df[column_to_be_imputed] = y_pred
    reduced_train_df[column_to_be_imputed] = reduced_y_train
    
    #imputing values into X_test
    y_pred_for_X_test = rfc.predict(X_test.drop(columns=[column_to_be_imputed]))
    X_test[column_to_be_imputed] = y_pred_for_X_test

    imputed_df=[reduced_train_df, reduced_test_df]
    imputed_df = pd.concat(imputed_df)
    return imputed_df

In [81]:
%%time
for column in cols_to_be_imputed_using_training:
    missing_data_df=impute_feature_by_training(missing_data_df, column)

Wall time: 3min 13s


In [82]:
print("Missing dataframe features with count of zero values")
print("-"*50)
print_columns_with_zero_values(missing_data_df, mixed_features_engineered_columns)

Missing dataframe features with count of zero values
--------------------------------------------------
caller_id_num  has  19  zero values
location_num  has  50  zero values
category_num  has  50  zero values
subcategory_num  has  73  zero values
closed_code_num  has  492  zero values
resolved_by_num  has  153  zero values


In [83]:
print("Test dataframe features with count of zero values")
print("-"*50)
print_columns_with_zero_values(X_test, mixed_features_engineered_columns)

Test dataframe features with count of zero values
--------------------------------------------------
caller_id_num  has  10  zero values
location_num  has  26  zero values
category_num  has  28  zero values
subcategory_num  has  38  zero values
closed_code_num  has  222  zero values
resolved_by_num  has  73  zero values


<h3>Observation</h3>

For remaining features we will use most frequent strategy and impute data.

In [84]:
missing_data_df.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_updated_by_num,location_num,category_num,...,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms,sys_created_by_num,u_symptom_num,opened_by_num,assignment_group_num,assigned_to_num
55379,0.517588,-0.105307,0.247074,1,1,2,2691,777,161,46,...,0,0,0,0,-0.165095,20,589,40,10,200
47879,-0.059340,-0.105307,-0.399533,1,1,2,5593,421,108,46,...,0,0,0,0,-0.641195,169,491,390,70,187
30919,-0.636268,-0.105307,-0.528854,1,1,2,4768,447,161,37,...,0,0,0,0,-0.865025,130,491,305,49,120
124692,-0.059340,-0.105307,-0.399533,1,1,2,1717,308,111,37,...,0,0,0,0,0.716414,70,491,157,46,83
17669,0.517588,-0.105307,-0.399533,1,1,2,2081,746,82,13,...,0,1,0,0,-1.018934,13,6,24,20,17


In [85]:
X_test.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_created_by_num,sys_updated_by_num,opened_by_num,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
83151,-0.059340,-0.105307,-0.140890,1,1,2,1932,167,241,8,...,1,1,0,0,1,0,0,0,0,-0.024584
101627,-0.059340,-0.105307,-0.528854,1,1,2,106,216,915,8,...,1,1,0,0,1,0,0,0,0,0.319830
64377,-0.636268,-0.105307,0.505717,1,1,2,2632,10,109,8,...,1,1,0,0,1,0,0,0,0,-0.271561
87520,-0.059340,-0.105307,-0.528854,1,1,2,4120,62,467,8,...,1,1,0,0,1,0,0,0,0,0.005142
128434,-0.636268,-0.105307,-0.658175,1,1,2,2651,52,908,8,...,1,1,0,0,1,0,0,0,0,0.756044


In [86]:
imp = SimpleImputer(missing_values=0, strategy="most_frequent")
imp.fit(missing_data_df)
missing_data_df = pd.DataFrame(data=imp.transform(missing_data_df), index=missing_data_df.index, columns = missing_data_df.columns)
X_test = pd.DataFrame(data=imp.transform(X_test), index=X_test.index, columns = X_test.columns)

In [87]:
dump_object_to_pickle(imp, "./git_repo/saved_models/most_frequent_simple_imputer_for_zero_value.pkl")

In [88]:
missing_data_df.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_updated_by_num,location_num,category_num,...,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms,sys_created_by_num,u_symptom_num,opened_by_num,assignment_group_num,assigned_to_num
55379,0.517588,-0.105307,0.247074,1,1,2,2691,777,161,46,...,1,1,1,1,-0.165095,20,589,40,10,200
47879,-0.05934,-0.105307,-0.399533,1,1,2,5593,421,108,46,...,1,1,1,1,-0.641195,169,491,390,70,187
30919,-0.636268,-0.105307,-0.528854,1,1,2,4768,447,161,37,...,1,1,1,1,-0.865025,130,491,305,49,120
124692,-0.05934,-0.105307,-0.399533,1,1,2,1717,308,111,37,...,1,1,1,1,0.716414,70,491,157,46,83
17669,0.517588,-0.105307,-0.399533,1,1,2,2081,746,82,13,...,1,1,1,1,-1.018934,13,6,24,20,17


In [89]:
X_test.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_created_by_num,sys_updated_by_num,opened_by_num,...,active_True,made_sla_True,contact_type_Email,contact_type_IVR,contact_type_Phone,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms
83151,-0.05934,-0.105307,-0.14089,1,1,2,1932,167,241,8,...,1,1,1,1,1,10,491,17,70,-0.024584
101627,-0.05934,-0.105307,-0.528854,1,1,2,106,216,915,8,...,1,1,1,1,1,10,491,17,70,0.31983
64377,-0.636268,-0.105307,0.505717,1,1,2,2632,10,109,8,...,1,1,1,1,1,10,491,17,70,-0.271561
87520,-0.05934,-0.105307,-0.528854,1,1,2,4120,62,467,8,...,1,1,1,1,1,10,491,17,70,0.005142
128434,-0.636268,-0.105307,-0.658175,1,1,2,2651,52,908,8,...,1,1,1,1,1,10,491,17,70,0.756044


In [90]:
print("Missing dataframe features with count of zero values")
print("-"*50)
print_columns_with_zero_values(missing_data_df, mixed_features_engineered_columns)

Missing dataframe features with count of zero values
--------------------------------------------------


In [91]:
print("Test dataframe features with count of zero values")
print("-"*50)
print_columns_with_zero_values(X_test, mixed_features_engineered_columns)

Test dataframe features with count of zero values
--------------------------------------------------


In [92]:
# https://stackoverflow.com/a/31844260/1225413
missing_data_df=missing_data_df.reindex(X_train.index)

In [93]:
missing_data_df.head(5)

,reassignment_count,reopen_count,sys_mod_count,impact,urgency,priority,caller_id_num,sys_updated_by_num,location_num,category_num,...,contact_type_Self service,knowledge_True,u_priority_confirmation_True,notify_Send Email,sys_updated_at_ms,sys_created_by_num,u_symptom_num,opened_by_num,assignment_group_num,assigned_to_num
12846,-0.636268,-0.105307,-0.658175,1,1,2,2001,214,204,37,...,1,1,1,1,-1.07693,52,4,108,25,58
16096,-0.636268,-0.105307,-0.658175,1,1,2,2895,908,204,40,...,1,1,1,1,-1.056181,34,444,59,70,192
52740,-0.05934,-0.105307,0.247074,1,1,2,1727,908,46,23,...,1,1,1,1,-0.141211,13,470,24,72,123
135624,-0.05934,4.695363,0.376395,1,1,2,4131,745,204,57,...,1,1,1,1,0.946789,171,491,305,56,167
55379,0.517588,-0.105307,0.247074,1,1,2,2691,777,161,46,...,1,1,1,1,-0.165095,20,589,40,10,200


<hr/>

<h1>Save Engineered Features</h1>

In [94]:
y_train

12846     77.727083
16096      6.919444
52740     16.013194
135624     9.829861
55379     18.429167
            ...    
41993      6.874306
97639      5.104167
95939      5.010417
117952     5.036806
43567     12.340278
Name: time_taken_to_complete, Length: 99198, dtype: float64

In [95]:
'''Generic method to save object in a file using pickle.'''
def save_pickle_to_obj(filename, obj):
    fileObj=open(filename,"wb")
    pickle.dump(obj,fileObj)
    fileObj.close()

In [96]:
save_pickle_to_obj("X_train_feature_enggineered_df.pkl",missing_data_df)
save_pickle_to_obj("y_train.pkl",y_train)
save_pickle_to_obj("X_test_feature_enggineered_df.pkl",X_test)
save_pickle_to_obj("y_test.pkl",y_test)

<hr  style="height:1px;"/>

<h1>Final Observations:</h1>


<ol>
    <li>Imputed only 1 category feature incident_state with most_frequent strategy.</li>
    <li>One hot encoded non-ordinal categorical features.</li>
    <li>Label encoded ordinal categorical features.</li>
    <li>For mixed features removed the repeative prefix part and kept only non-repetitive numerical portion.</li>
    <li>Converted temporal features to time in milliseconds.</li>
    <li>Standarized all numerical feature using StandardScaler which will help in convergence faster for few of ML Algos.</li>
    <li>Imputed values for mixed features columns with more than 3% of missing values using RandomForestClassifer.</li>
    <li>Imputed values for mixed features columns with less than 3% of missing values using most frequent values.</li>
</ol>